# Prediction

The aim of this phase is to combine all previous steps into one file and predict 'delayInSecods' fo the test file

In [1]:
# import libraries
import numpy as np
np.random.seed(2018)
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import xgboost as xgb 
from sklearn.ensemble import RandomForestRegressor

### Test file preprocessing

In [2]:
# set column types to minimize filed size
column_types= {'id': 'int32',
               'stopId': 'int32',
               'routeId': 'int16',
               'vehicleId': 'int32',
               'tripId': 'int16',
               'delayInSeconds': 'int32',
               'agencyId': 'int8',
               }
# create DataFrame
df = pd.read_csv('train.csv', dtype=column_types, parse_dates=['delayPredictionTimestamp','scheduleTime'])
# merge stopsintrips file.Merge stopSequence on:'routeId','stopId', 'tripId', merge agency ID on route ID only to have minimal number of missing rows
#Topology excluced as contains same info as route ID
stops=pd.read_csv('stopsintrips.csv', dtype=column_types)
stops_1=stops[['routeId','agencyId']].drop_duplicates()
df=df.merge(right=stops_1, how='left', on=['routeId'])
df=df.merge(right=stops[['routeId', 'stopId', 'stopSequence', 'tripId']], how='left', on=['routeId','stopId', 'tripId'])
# merge stops file containig location. Not all locations exist in stops file.
location=pd.read_csv('stops.csv', dtype=column_types)
missing_location=df[~df['stopId'].isin(location['stopId'])]['stopId'].unique().reshape(-1,1)
# as there is linear correlation between location and stopID, missing data to be predicted by LinearRegression model
# predict missing stopLat
from sklearn.linear_model import LinearRegression
X=location.iloc[:,:1].values
y=location['stopLat'].values
lr_lat=LinearRegression()
lr_lat.fit(X,y)
# predict missing stopLon
X=location.iloc[:,:1].values
y=location['stopLon'].values
lr_lon=LinearRegression()
lr_lon.fit(X,y)
# add predicted missing locations to location df
missing_lon=lr_lon.predict(missing_location)
missing_lat=lr_lat.predict(missing_location)
missing_loc_df=pd.DataFrame({'stopId':np.squeeze(missing_location), 'stopLat':missing_lat, 'stopLon':missing_lon})
location=pd.concat([location,missing_loc_df],axis=0)
# merge locations with df
df=df.merge(right=location, how='left', left_on=['stopId'], right_on=['stopId'])
# merge weather df. Remove duplicates. Forward fill missing hour rows. Drop columns that do not add value (tested)
weather=pd.read_csv('weatherHistory.csv',sep=';')
weather['dt']=pd.to_datetime(weather['dt'],unit='s')
weather.drop_duplicates(subset='dt', keep='first',inplace=True)
weather=weather.set_index(pd.DatetimeIndex(weather['dt']))
weather=weather.resample('H').ffill()
weather.drop(['dt','dt_iso', 'city_id', 'temp_min', 'temp_max','weather_icon','weather_description','weather_id', 'weather_main', 'pressure', 'clouds_all'], axis=1, inplace=True)
cols=['humidity','wind_speed','wind_deg']
for col in cols: weather[col]=pd.to_numeric(weather[col], downcast='signed')
df['dt']=df['scheduleTime'].dt.round('h')
df=df.merge(right=weather, how='left', left_on='dt', right_index=True)
df.drop(['dt'], axis=1, inplace=True)
# deal with other missing data. AgencyID: fill with most common value '1', stopSequence: fill with median '12'
df['agencyId']=df['agencyId'].fillna(1)
df['stopSequence']=df['stopSequence'].fillna(12)

### Test file preprocessing

In [12]:
# Create test DataFrame
test = pd.read_csv('test.csv', dtype=column_types, parse_dates=['delayPredictionTimestamp','scheduleTime'])
# merge stopsintrips file
test=test.merge(right=stops_1, how='left', on=['routeId'])
test=test.merge(right=stops[['routeId', 'stopId', 'stopSequence', 'tripId']], how='left', on=['routeId','stopId', 'tripId'])
# find missing stopsID in file with location
missing_location=test[~test['stopId'].isin(location['stopId'])]['stopId'].unique().reshape(-1,1)
# predict missing location with already trained lr models
missing_lon=lr_lon.predict(missing_location)
missing_lat=lr_lat.predict(missing_location)
missing_loc_df=pd.DataFrame({'stopId':np.squeeze(missing_location), 'stopLat':missing_lat, 'stopLon':missing_lon})
location=pd.concat([location,missing_loc_df],axis=0)
# merge file with location
test=test.merge(right=location, how='left', left_on=['stopId'], right_on=['stopId'])
# merge weather file
test['dt']=test['scheduleTime'].dt.round('h')
test=test.merge(right=weather, how='left', left_on='dt', right_index=True, copy=False)
test.drop(['dt'], axis=1, inplace=True)
# fill missing data
test['agencyId']=test['agencyId'].fillna(1)
test['stopSequence']=test['stopSequence'].fillna(12)

### Feature engineering

In [19]:
# difference between delayPrediction and schedule time. Wrong date when time close to midnight need to add or subtract 24h
df['time_diff']=(df['delayPredictionTimestamp']-df['scheduleTime']).astype('timedelta64[s]')
df['time_diff']=pd.to_numeric(df['time_diff'], downcast='signed')
df['time_diff']=df['time_diff'].apply(lambda x: x+86400 if x <-50000 else x)
df['time_diff']=df['time_diff'].apply(lambda x: x-86400 if x >50000 else x)
# time features: hour, dayofweek, time- day time in seconds
df['hour'] = df['scheduleTime'].dt.hour
df['dayofweek'] = df['scheduleTime'].dt.dayofweek
df['time']=df['delayPredictionTimestamp'].dt.second+df['delayPredictionTimestamp'].dt.minute*60+df['delayPredictionTimestamp'].dt.hour*3600
# mean delayInSeconds of each agency, routeId, stopID, vehicleId, tripID
agencyMeanDelay = df[ ['agencyId', 'delayInSeconds'] ].groupby(['agencyId']).mean().to_dict()['delayInSeconds']
df['agencyMeanDelay'] = df['agencyId'].map(lambda x: agencyMeanDelay[x])
routeMeanDelay = df[ ['routeId', 'delayInSeconds'] ].groupby(['routeId']).mean().to_dict()['delayInSeconds']
df['routeMeanDelay'] = df['routeId'].map(lambda x: routeMeanDelay[x])
stopIdMeanDelay = df[ ['stopId', 'delayInSeconds'] ].groupby(['stopId']).mean().to_dict()['delayInSeconds']
df['stopIdMeanDelay'] = df['stopId'].map(lambda x: stopIdMeanDelay[x])
vehicleIdMeanDelay = df[ ['vehicleId', 'delayInSeconds'] ].groupby(['vehicleId']).mean().to_dict()['delayInSeconds']
df['vehicleIdMeanDelay'] = df['vehicleId'].map(lambda x: vehicleIdMeanDelay[x])
tripIdMeanDelay = df[ ['tripId', 'delayInSeconds'] ].groupby(['tripId']).mean().to_dict()['delayInSeconds']
df['tripIdMeanDelay'] = df['tripId'].map(lambda x: tripIdMeanDelay[x])

In [32]:
# define features function to exclude some columns from features matrix
def features(df):
    feats = df.columns.values
    black_list = ['id', 'delayInSeconds','delayPredictionTimestamp','scheduleTime', 'delayInSeconds_xgb']
    return [feat for feat in feats if feat not in black_list]

In [21]:
# check feature engineering
df[features].head()

,stopId,routeId,vehicleId,tripId,agencyId,stopSequence,stopLat,stopLon,temp,humidity,...,wind_deg,time_diff,hour,dayofweek,time,agencyMeanDelay,routeMeanDelay,stopIdMeanDelay,vehicleIdMeanDelay,tripIdMeanDelay
0,204,11,377,22,2.0,37.0,54.39919,18.59498,291.15,59,...,250,92,12,2,43832,163.450028,153.355873,465.887850,146.086857,171.803197
1,2051,11,377,22,2.0,31.0,54.41812,18.57758,291.15,59,...,250,-47,12,2,43153,163.450028,153.355873,114.189903,146.086857,171.803197
2,2092,11,377,22,2.0,36.0,54.40425,18.59102,291.15,59,...,250,64,12,2,43684,163.450028,153.355873,156.111310,146.086857,171.803197
3,2094,11,377,22,2.0,35.0,54.40905,18.58866,291.15,59,...,250,8,12,2,43568,163.450028,153.355873,119.286389,146.086857,171.803197
4,2096,11,377,22,2.0,34.0,54.41262,18.58775,291.15,59,...,250,21,12,2,43521,163.450028,153.355873,79.494773,146.086857,171.803197


### Model training

I would train both XGBoost model and RandomForest one with the best parameters I chose in model tuning phase. Then I'll check predicted 'delayInSeconds' obtained from both models and combination of both models in Kaggle competition and choose the best final model.

In [22]:
# feature matrix, labels array creation
X=df[features].values
y=df['delayInSeconds'].values

In [23]:
# train XGBoost model on the best parameters chosen in grid search
xgb_model = xgb.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=300, random_state=2018, n_jobs=-1)

In [24]:
# fit model
xgb_model.fit(X,y)

[08:37:50] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=2018,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [25]:
# train Random Forest model on the best parameters chosen in grid search
rf=RandomForestRegressor(max_depth=12, min_samples_leaf=5, n_estimators=50, random_state=2018, n_jobs=-1)

In [26]:
# fit model
rf.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
           oob_score=False, random_state=2018, verbose=0, warm_start=False)

### Prediction

In [27]:
# Feature engineering of test DataFrame
# time_diff
test['time_diff']=(test['delayPredictionTimestamp']-test['scheduleTime']).astype('timedelta64[s]')
test['time_diff']=pd.to_numeric(test['time_diff'], downcast='signed')
test['time_diff']=test['time_diff'].apply(lambda x: x+86400 if x <-50000 else x)
test['time_diff']=test['time_diff'].apply(lambda x: x-86400 if x >50000 else x)
# time features
test['hour'] = test['scheduleTime'].dt.hour
test['dayofweek'] = test['scheduleTime'].dt.dayofweek
test['time']=test['delayPredictionTimestamp'].dt.second+test['delayPredictionTimestamp'].dt.minute*60+test['delayPredictionTimestamp'].dt.hour*3600
# mean delayInSec, if no data in mean dictionary, use mean delay of whole training dataset
meanDelay=df['delayInSeconds'].mean()
test['agencyMeanDelay'] = test['agencyId'].map(lambda x: agencyMeanDelay.get(x,meanDelay))
test['routeMeanDelay'] = test['routeId'].map(lambda x: routeMeanDelay.get(x,meanDelay))
test['stopIdMeanDelay'] = test['stopId'].map(lambda x: stopIdMeanDelay.get(x,meanDelay))
test['vehicleIdMeannDelay'] = test['vehicleId'].map(lambda x: vehicleIdMeanDelay.get(x,meanDelay))
test['tripIdMeanDelay'] = test['tripId'].map(lambda x: tripIdMeanDelay.get(x,meanDelay))

In [28]:
# predict delayInSeconds using XGBoost model
test['delayInSeconds_xgb']=xgb_model.predict(test[features].values)

In [29]:
# change predictions to integers
test['delayInSeconds_xgb']=test['delayInSeconds_xgb'].astype('int')

In [33]:
# predict delayInSeconds using Random Forest model
test['delayInSeconds_rf']=rf.predict(test[features].values)

In [34]:
# change predictions to integers
test['delayInSeconds_rf']=test['delayInSeconds_rf'].astype('int')

In [35]:
# take weighted average from 2 models predictions as final prediction
test['delayInSeconds']=0.7*test['delayInSeconds_xgb']+0.3*test['delayInSeconds_rf']

In [40]:
# save prediction file
test[['id','delayInSeconds']].to_csv('solution_xgb&rf_final', index=False)

I tested each model separately and the a few combination of both models. I got following results on Kaggle:

The combination 70% of values predicted by XGBoost and 30% of values predicted by RandomForest was my best prediction and the one that won the competition.